In [1]:
import pandas
from geosnap import Community
from geosnap import datasets
import fiona
import geopandas
%matplotlib inline
import matplotlib.pyplot as plt
import segregation

# questions for today 5/27
# bucket problem with geosnap?
# losing district and county name with frpm aggregation
# merge operation breaking ela currstatus 

C:\Users\fiend\Anaconda2\envs\capstone\lib\site-packages\geosnap\_data.py:123: UserWarning: Unable to locate local census data. Streaming instead.
If you plan to use census data repeatedly you can store it locally with the io.store_census function for better performance
  "Unable to locate local census data. Streaming instead.\n"
Loading manifest: 100%|██████████| 5/5 [00:00<00:00, 2506.16entries/s]
C:\Users\fiend\Anaconda2\envs\capstone\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.cluster.spectral module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.cluster. Anything that cannot be imported from sklearn.cluster is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
pandas.set_option('display.max_rows', 500)
pandas.set_option('display.max_column', 500)

In [3]:
ele = geopandas.read_file("districts_tiger/tl_2015_us_sde.shp")
# Reading in shapefiles from census/tiger

In [4]:
seco = geopandas.read_file("districts_tiger/tl_2015_us_sds.shp")

In [5]:
uni = geopandas.read_file("districts_tiger/tl_2015_us_sdu.shp")

In [6]:
ele = ele.loc[ele['STATEFP'] == '06']
# slicing out California
# elementary school districts in California

In [7]:
seco = seco.loc[seco['STATEFP'] == '06']
# secondary school districts in California

In [8]:
uni = uni.loc[uni['STATEFP'] == '06']
# unified school districts in California

In [9]:
len(ele)+len(seco)+len(uni)

986

## Combining shapefiles into single geodataframe

In [10]:
ele['origin'] = 'elementary'

In [11]:
seco['origin'] = 'secondary'

In [12]:
uni['origin'] = 'unified'

In [13]:
adistricts = uni.append([ele, seco])
#all districts

In [14]:
adistricts = adistricts.sort_values('NAME', ascending=True)

In [15]:
check = adistricts['NAME'].str.contains('County Office')

res = [i for i, val in enumerate(check) if val]
print(res)

[]


In [157]:
cnames = (
'Alameda County', 
'Alpine County', 
'Amador County', 
'Butte County', 
'Calaveras County', 
'Colusa County', 
'Contra Costa County',
'Del Norte County', 
'El Dorado County', 
'Fresno County', 
'Glenn County',
'Humboldt County',
'Imperial County', 
'Inyo County',
'Kern County', 
'Kings County',
'Lake County', 
'Lassen County',
'Los Angeles County', 
'Madera County', 
'Marin County', 
'Mariposa County', 
'Mendocino County', 
'Merced County', 
'Modoc County', 
'Mono County', 
'Monterey County', 
'Napa County', 
'Nevada County', 
'Orange County',
'Placer County',
'Plumas County', 
'Riverside County', 
'Sacramento County', 
'San Benito County', 
'San Bernardino County', 
'San Diego County', 
'San Francisco County', 
'San Joaquin County', 
'San Luis Obispo County',
'San Mateo County',
'Santa Barbara County', 
'Santa Clara County',
'Santa Cruz County', 
'Shasta County', 
'Sierra County', 
'Siskiyou County', 
'Solano County', 
'Sonoma County',
'Stanislaus County', 
'Sutter County', 
'Tehama County',
'Trinity County', 
'Tulare County', 
'Tuolumne County', 
'Ventura County', 
'Yolo County',
'Yuba County')
cnames

('Alameda County',
 'Alpine County',
 'Amador County',
 'Butte County',
 'Calaveras County',
 'Colusa County',
 'Contra Costa County',
 'Del Norte County',
 'El Dorado County',
 'Fresno County',
 'Glenn County',
 'Humboldt County',
 'Imperial County',
 'Inyo County',
 'Kern County',
 'Kings County',
 'Lake County',
 'Lassen County',
 'Los Angeles County',
 'Madera County',
 'Marin County',
 'Mariposa County',
 'Mendocino County',
 'Merced County',
 'Modoc County',
 'Mono County',
 'Monterey County',
 'Napa County',
 'Nevada County',
 'Orange County',
 'Placer County',
 'Plumas County',
 'Riverside County',
 'Sacramento County',
 'San Benito County',
 'San Bernardino County',
 'San Diego County',
 'San Francisco County',
 'San Joaquin County',
 'San Luis Obispo County',
 'San Mateo County',
 'Santa Barbara County',
 'Santa Clara County',
 'Santa Cruz County',
 'Shasta County',
 'Sierra County',
 'Siskiyou County',
 'Solano County',
 'Sonoma County',
 'Stanislaus County',
 'Sutter County

In [158]:
geoid = ('06001',
'06003',
'06005',
'06007',
'06009',
'06011',
'06013',
'06015',
'06017',
'06019',
'06021',
'06023',
'06025',
'06027',
'06029',
'06031',
'06033',
'06035',
'06037',
'06039',
'06041',
'06043',
'06045',
'06047',
'06049',
'06051',
'06053',
'06055',
'06057',
'06059',
'06061',
'06063',
'06065',
'06067',
'06069',
'06071',
'06073',
'06075',
'06077',
'06079',
'06081',
'06083',
'06085',
'06087',
'06089',
'06091',
'06093',
'06095',
'06097',
'06099',
'06101',
'06103',
'06105',
'06107',
'06109',
'06111',
'06113', 
'06115')
geoid

('06001',
 '06003',
 '06005',
 '06007',
 '06009',
 '06011',
 '06013',
 '06015',
 '06017',
 '06019',
 '06021',
 '06023',
 '06025',
 '06027',
 '06029',
 '06031',
 '06033',
 '06035',
 '06037',
 '06039',
 '06041',
 '06043',
 '06045',
 '06047',
 '06049',
 '06051',
 '06053',
 '06055',
 '06057',
 '06059',
 '06061',
 '06063',
 '06065',
 '06067',
 '06069',
 '06071',
 '06073',
 '06075',
 '06077',
 '06079',
 '06081',
 '06083',
 '06085',
 '06087',
 '06089',
 '06091',
 '06093',
 '06095',
 '06097',
 '06099',
 '06101',
 '06103',
 '06105',
 '06107',
 '06109',
 '06111',
 '06113',
 '06115')

In [160]:
cfips = pandas.DataFrame(zip(cnames, geoid), columns=['county_name', 'GEOID'])
cfips

,county_name,GEOID
0,Alameda County,06001
1,Alpine County,06003
2,Amador County,06005
3,Butte County,06007
4,Calaveras County,06009
5,Colusa County,06011
6,Contra Costa County,06013
7,Del Norte County,06015
8,El Dorado County,06017
9,Fresno County,06019


In [132]:
adistricts['GEOID'].astype(str)
adistricts

,STATEFP,UNSDLEA,GEOID,NAME,LSAD,LOGRADE,HIGRADE,MTFCC,SDTYP,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry,origin,ELSDLEA,SCSDLEA
645,06,01620,0601620,ABC Unified School District,00,KG,12,G5420,None,E,38772821,556042,+33.8637711,-118.0665456,"POLYGON ((-118.10857 33.88775, -118.10843 33.8...",unified,NaN,NaN
29,06,NaN,0601650,Acalanes Union High School District,00,09,12,G5410,None,E,205137702,4413448,+37.8772150,-122.1288576,"POLYGON ((-122.26126 37.92143, -122.26009 37.9...",secondary,NaN,01650
403,06,NaN,0601680,Ackerman Elementary School District,00,KG,08,G5400,None,E,11308882,7628,+38.9366318,-121.0507823,"POLYGON ((-121.07045 38.92923, -121.07045 38.9...",elementary,01680,NaN
658,06,00001,0600001,Acton-Agua Dulce Unified School District,00,KG,12,G5420,None,E,555175128,182565,+34.4509057,-118.2276141,"POLYGON ((-118.38010 34.50369, -118.37724 34.5...",unified,NaN,NaN
376,06,NaN,0601710,Adelanto Elementary School District,00,KG,06,G5400,None,E,868470886,832474,+34.6306380,-117.5205628,"POLYGON ((-117.66785 34.79074, -117.66784 34.7...",elementary,01710,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124,06,NaN,0606002,Yosemite Unified School District in Raymond-Kn...,00,09,12,G5410,A,F,502931732,11866102,+37.1737072,-119.9030808,"POLYGON ((-120.05463 37.17719, -120.05462 37.1...",secondary,NaN,06002
335,06,NaN,0643380,Yreka Union Elementary School District,00,KG,08,G5400,None,E,417267221,1100011,+41.7728486,-122.6949023,"POLYGON ((-122.79336 41.83972, -122.79325 41.8...",elementary,43380,NaN
98,06,NaN,0643410,Yreka Union High School District,00,09,12,G5410,None,E,3355267245,20958376,+41.7984998,-122.6322692,"POLYGON ((-123.16027 41.78169, -123.16030 41.7...",secondary,NaN,43410
545,06,43470,0643470,Yuba City Unified School District,00,KG,12,G5420,None,E,294574293,4783349,+39.0342246,-121.6633133,"POLYGON ((-121.77132 39.09872, -121.77086 39.0...",unified,NaN,NaN


In [137]:
test = adistricts
test

,STATEFP,UNSDLEA,GEOID,NAME,LSAD,LOGRADE,HIGRADE,MTFCC,SDTYP,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry,origin,ELSDLEA,SCSDLEA
645,06,01620,0601620,ABC Unified School District,00,KG,12,G5420,None,E,38772821,556042,+33.8637711,-118.0665456,"POLYGON ((-118.10857 33.88775, -118.10843 33.8...",unified,NaN,NaN
29,06,NaN,0601650,Acalanes Union High School District,00,09,12,G5410,None,E,205137702,4413448,+37.8772150,-122.1288576,"POLYGON ((-122.26126 37.92143, -122.26009 37.9...",secondary,NaN,01650
403,06,NaN,0601680,Ackerman Elementary School District,00,KG,08,G5400,None,E,11308882,7628,+38.9366318,-121.0507823,"POLYGON ((-121.07045 38.92923, -121.07045 38.9...",elementary,01680,NaN
658,06,00001,0600001,Acton-Agua Dulce Unified School District,00,KG,12,G5420,None,E,555175128,182565,+34.4509057,-118.2276141,"POLYGON ((-118.38010 34.50369, -118.37724 34.5...",unified,NaN,NaN
376,06,NaN,0601710,Adelanto Elementary School District,00,KG,06,G5400,None,E,868470886,832474,+34.6306380,-117.5205628,"POLYGON ((-117.66785 34.79074, -117.66784 34.7...",elementary,01710,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124,06,NaN,0606002,Yosemite Unified School District in Raymond-Kn...,00,09,12,G5410,A,F,502931732,11866102,+37.1737072,-119.9030808,"POLYGON ((-120.05463 37.17719, -120.05462 37.1...",secondary,NaN,06002
335,06,NaN,0643380,Yreka Union Elementary School District,00,KG,08,G5400,None,E,417267221,1100011,+41.7728486,-122.6949023,"POLYGON ((-122.79336 41.83972, -122.79325 41.8...",elementary,43380,NaN
98,06,NaN,0643410,Yreka Union High School District,00,09,12,G5410,None,E,3355267245,20958376,+41.7984998,-122.6322692,"POLYGON ((-123.16027 41.78169, -123.16030 41.7...",secondary,NaN,43410
545,06,43470,0643470,Yuba City Unified School District,00,KG,12,G5420,None,E,294574293,4783349,+39.0342246,-121.6633133,"POLYGON ((-121.77132 39.09872, -121.77086 39.0...",unified,NaN,NaN


In [140]:
test['GEOID'] = test['GEOID'].astype(str).str[:-2]
test

,STATEFP,UNSDLEA,GEOID,NAME,LSAD,LOGRADE,HIGRADE,MTFCC,SDTYP,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry,origin,ELSDLEA,SCSDLEA
645,06,01620,06016,ABC Unified School District,00,KG,12,G5420,None,E,38772821,556042,+33.8637711,-118.0665456,"POLYGON ((-118.10857 33.88775, -118.10843 33.8...",unified,NaN,NaN
29,06,NaN,06016,Acalanes Union High School District,00,09,12,G5410,None,E,205137702,4413448,+37.8772150,-122.1288576,"POLYGON ((-122.26126 37.92143, -122.26009 37.9...",secondary,NaN,01650
403,06,NaN,06016,Ackerman Elementary School District,00,KG,08,G5400,None,E,11308882,7628,+38.9366318,-121.0507823,"POLYGON ((-121.07045 38.92923, -121.07045 38.9...",elementary,01680,NaN
658,06,00001,06000,Acton-Agua Dulce Unified School District,00,KG,12,G5420,None,E,555175128,182565,+34.4509057,-118.2276141,"POLYGON ((-118.38010 34.50369, -118.37724 34.5...",unified,NaN,NaN
376,06,NaN,06017,Adelanto Elementary School District,00,KG,06,G5400,None,E,868470886,832474,+34.6306380,-117.5205628,"POLYGON ((-117.66785 34.79074, -117.66784 34.7...",elementary,01710,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124,06,NaN,06060,Yosemite Unified School District in Raymond-Kn...,00,09,12,G5410,A,F,502931732,11866102,+37.1737072,-119.9030808,"POLYGON ((-120.05463 37.17719, -120.05462 37.1...",secondary,NaN,06002
335,06,NaN,06433,Yreka Union Elementary School District,00,KG,08,G5400,None,E,417267221,1100011,+41.7728486,-122.6949023,"POLYGON ((-122.79336 41.83972, -122.79325 41.8...",elementary,43380,NaN
98,06,NaN,06434,Yreka Union High School District,00,09,12,G5410,None,E,3355267245,20958376,+41.7984998,-122.6322692,"POLYGON ((-123.16027 41.78169, -123.16030 41.7...",secondary,NaN,43410
545,06,43470,06434,Yuba City Unified School District,00,KG,12,G5420,None,E,294574293,4783349,+39.0342246,-121.6633133,"POLYGON ((-121.77132 39.09872, -121.77086 39.0...",unified,NaN,NaN


In [142]:
test2 = pandas.DataFrame(countyfips ,columns=['county_name', 'GEOID',])
test2

ValueError: Shape of passed values is (115, 1), indices imply (115, 2)

In [17]:
adsgeo = adistricts['NAME']
# all districs name list

In [18]:
adistricts.shape

(986, 18)

In [19]:
items = pandas.DataFrame(columns=['District Code', 'districtname', 'studentgroup', 'currstatus', 'priorstatus', 'change']) 
items

,District Code,districtname,studentgroup,currstatus,priorstatus,change


In [20]:
adsgeo.unique().shape
# todo: some districts listed twice

(970,)

In [21]:
dupes = adistricts[adistricts['NAME'].isin(adistricts['NAME'].value_counts()[adistricts['NAME'].value_counts()>2].index)]
print(dupes['NAME'].unique())
# 3 Jefferson
# 3 Lakeside
# 3 Pioneer

# create new column indicating countyID

['Jefferson Elementary School District'
 'Lakeside Union Elementary School District'
 'Pioneer Union Elementary School District']


### Harmonizing data

ela + math `currstatus` Average distance from Standard of students who took the Smarter Balanced summative assessment in ela/math


options<br>
* questions of differences across groups? (no geography)
* differences across districts (group by district)
* differences across groups in districts (interaction effects > more pronounced differences in certain districts)

visualization

* what is the best indicator to use for choropleth ()

* or - use state level values as benchmark for each group (how is it reflected in each district, heterogeniety in space?)

## ELA acheivement indicator data
Find the codebook [here](https://www.cde.ca.gov/ta/ac/cm/ela18.asp)

In [22]:
ela = pandas.read_csv('lcff_data/eladownload2018.csv')
ela.shape

C:\Users\fiend\Anaconda2\envs\capstone\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (6,23) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(148838, 32)

In [23]:
ela.head()

,cds,rtype,schoolname,districtname,countyname,charter_flag,coe_flag,dass_flag,studentgroup,currdenom,currdenom_swd,currstatus,priordenom,priordenom_swd,priorstatus,change,statuslevel,changelevel,color,box,hscutpoints,curradjustment,prioradjustment,pairshare_method,caa_denom,caa_level1_num,caa_level1_pct,caa_level2_num,caa_level2_pct,caa_level3_num,caa_level3_pct,ReportingYear
0,0,X,NaN,NaN,NaN,NaN,NaN,NaN,AA,172224,NaN,-51.8,177009.0,NaN,-52.7,0.9,2,3,2,180,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018
1,0,X,NaN,NaN,NaN,NaN,NaN,NaN,AI,16662,NaN,-36.8,17234.0,NaN,-39.1,2.3,2,3,2,180,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018
2,0,X,NaN,NaN,NaN,NaN,NaN,NaN,ALL,3171704,NaN,-6.0,3200441.0,NaN,-8.2,2.2,2,3,2,180,NaN,NaN,NaN,NaN,36140.0,18581.0,51.4,11330.0,31.4,6229.0,17.2,2018
3,0,X,NaN,NaN,NaN,NaN,NaN,NaN,AS,293002,NaN,62.4,295040.0,NaN,60.5,2.0,5,3,5,30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018
4,0,X,NaN,NaN,NaN,NaN,NaN,NaN,EL,1000703,NaN,-47.1,1021418.0,NaN,-50.4,3.3,2,4,3,190,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018


## Creating state-level variable

In [24]:
replacements = dict(
    studentgroup = {'ALL':'all_students',
                        'AA':'african_american',
                        'AI':'american_indian',
                        'AS':'asian',
                        'FI':'filipino',
                        'HI':'hispanic',
                        'PI':'pacific_islander',
                        'WH':'white',
                        'MR':'multiple_races',
                        'EL':'english_learners',
                        'ELO':'english_learners_only',
                        'RFP':'rfep_only',
                        'EO':'english_only',
                        'SED':'ses_disad',
                        'SWD':'disabilities',
                        'FOS':'foster_youth',
                        'HOM':'homeless_youth',
                })
# dictionary for replacing acronyms, reuse later

ela['studentgroup'].astype(str)
ela = ela.replace(replacements)
# replace acronyms

j = ela
j = j.loc[j['rtype'] == 'X']
# select by no-type (state)
k = pandas.concat([j.pop(x) for x in ['studentgroup', 
                                      'currstatus', 
                                      'priorstatus', 
                                      'change',
                                      'currdenom']],
                  1)
# slice out columns
ela_state = k.sort_values(by='currstatus', ascending=False)
# state-level ela performance variable

## Aggregation 

In [25]:
# fixing cds codes to match with other datasets
ela['cds'].astype(str)
# changing cds code to string
ela = ela[ela.coe_flag !='Y']
# removing county offices of education
ela = ela.loc[ela['rtype'] == 'D']
# select district level entries
ela['cds'] = ela['cds'].apply(lambda x: '{0:0>14}'.format(x))
# adding leading zero to 13 digit int
ela['District Code'] = [x[2:7] for x in ela['cds']]
# adding district only code
elat = set(ela['cds'])
# creating test set variable
ela.head()

,cds,rtype,schoolname,districtname,countyname,charter_flag,coe_flag,dass_flag,studentgroup,currdenom,currdenom_swd,currstatus,priordenom,priordenom_swd,priorstatus,change,statuslevel,changelevel,color,box,hscutpoints,curradjustment,prioradjustment,pairshare_method,caa_denom,caa_level1_num,caa_level1_pct,caa_level2_num,caa_level2_pct,caa_level3_num,caa_level3_pct,ReportingYear,District Code
164,01611190000000,D,NaN,Alameda Unified,Alameda,NaN,NaN,NaN,african_american,298,NaN,-43.0,332.0,NaN,-48.7,5.7,2,4,3,190,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018,61119
165,01611190000000,D,NaN,Alameda Unified,Alameda,NaN,NaN,NaN,american_indian,13,NaN,-14.8,14.0,NaN,14.3,-29.1,2,1,0,0,NaN,NaN,-0.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018,61119
166,01611190000000,D,NaN,Alameda Unified,Alameda,NaN,NaN,NaN,all_students,4550,NaN,36.8,4541.0,NaN,32.8,4.0,4,4,4,90,NaN,NaN,NaN,NaN,38.0,26.0,68.4,6.0,15.8,6.0,15.8,2018,61119
167,01611190000000,D,NaN,Alameda Unified,Alameda,NaN,NaN,NaN,asian,1288,NaN,48.6,1334.0,NaN,54.0,-5.3,5,2,4,20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018,61119
168,01611190000000,D,NaN,Alameda Unified,Alameda,NaN,NaN,NaN,english_learners,1084,NaN,-2.5,1176.0,NaN,-1.4,-1.1,3,3,3,130,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018,61119


In [26]:
# Create an empty list 
Row_list =[] 
  
# Iterate over each row 
for index, rows in ela.iterrows(): 
    # Create list for the current row 
    my_list =[rows.studentgroup, rows.currstatus, rows.priorstatus, rows.change, rows.currdenom] 
      
    # append the list to the final list 
    Row_list.append(my_list) 
  
 
Row_list

[['african_american', -43.0, -48.7, 5.7, 298],
 ['american_indian', -14.8, 14.3, -29.1, 13],
 ['all_students', 36.8, 32.8, 4.0, 4550],
 ['asian', 48.6, 54.0, -5.3, 1288],
 ['english_learners', -2.5, -1.4, -1.1, 1084],
 ['english_learners_only', -42.0, -44.7, 2.7, 643],
 ['english_only', 43.5, 38.3, 5.2, 3025],
 ['filipino', 29.8, 16.0, 13.9, 275],
 ['foster_youth', nan, nan, nan, 9],
 ['hispanic', 5.0, 0.9, 4.1, 720],
 ['homeless_youth', -75.5, -56.7, -18.7, 30],
 ['multiple_races', 46.6, 40.4, 6.2, 555],
 ['pacific_islander', -27.3, -49.1, 21.8, 40],
 ['rfep_only', 55.0, 55.5, -0.5, 441],
 ['ses_disad', -13.1, -20.0, 6.9, 1491],
 ['disabilities', -73.1, -80.4, 7.3, 506],
 ['white', 60.8, 54.1, 6.7, 1346],
 ['african_american', -9.6, 12.2, -21.8, 53],
 ['american_indian', nan, nan, nan, 1],
 ['all_students', 57.5, 64.1, -6.6, 1887],
 ['asian', 73.0, 73.7, -0.8, 513],
 ['english_learners', 31.0, 37.6, -6.6, 449],
 ['english_learners_only', -11.4, -12.5, 1.1, 186],
 ['english_only', 60.9

In [27]:
#creating district-level ela performance variable (dictionary)
dfe = ela
dfe = pandas.concat([dfe.pop(x) for x in ['District Code',
                                          'districtname',                                          
                                          'studentgroup', 
                                          'currstatus', 
                                          'priorstatus', 
                                          'change']],
                    1)
# slice out columns

## Math achievement indicator data
find the codebook [here](https://www.cde.ca.gov/ta/ac/cm/math18.asp)

In [28]:
math = pandas.read_csv('lcff_data/mathdownload2018.csv')
math.shape

C:\Users\fiend\Anaconda2\envs\capstone\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (6,23) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(148705, 32)

In [29]:
math['studentgroup'].astype(str)
math = math.replace(replacements)
# replace codes
p = math
p = p.loc[p['rtype'] == 'X']
# select by no-type (state)
q = pandas.concat([p.pop(x) for x in ['studentgroup', 
                                      'currstatus', 
                                      'priorstatus', 
                                      'change',
                                      'currdenom']],
                  1)
# slice out columns
math_state = q.sort_values(by='currstatus', ascending=False)


## Aggregation

In [30]:
# fixing cds codes to match with other datasets
math['cds'].astype(str)
# changing to string
math = math[math.coe_flag !='Y']
# removing county offices of education
math = math.loc[math['rtype'] == 'D']
# select district level performance
math['cds'] = math['cds'].apply(lambda x: '{0:0>14}'.format(x))
# adding leading zero to 13 digit int
math['District Code'] = [x[2:7] for x in math['cds']]
# adding district only code

In [31]:
math.head()

,cds,rtype,schoolname,districtname,countyname,charter_flag,coe_flag,dass_flag,studentgroup,currdenom,currdenom_swd,currstatus,priordenom,priordenom_swd,priorstatus,change,statuslevel,changelevel,color,box,hscutpoints,curradjustment,prioradjustment,pairshare_method,caa_denom,caa_level1_num,caa_level1_pct,caa_level2_num,caa_level2_pct,caa_level3_num,caa_level3_pct,ReportingYear,District Code
163,01611190000000,D,NaN,Alameda Unified,Alameda,NaN,NaN,NaN,african_american,297,NaN,-83.9,332.0,NaN,-82.8,-1.1,2,3,2,180,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018,61119
164,01611190000000,D,NaN,Alameda Unified,Alameda,NaN,NaN,NaN,american_indian,13,NaN,-51.8,14.0,NaN,-50.0,-1.8,2,3,0,0,NaN,NaN,-0.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018,61119
165,01611190000000,D,NaN,Alameda Unified,Alameda,NaN,NaN,NaN,all_students,4543,NaN,14.3,4547.0,NaN,10.8,3.4,4,4,4,90,NaN,NaN,NaN,NaN,38.0,30.0,78.9,6.0,15.8,2.0,5.3,2018,61119
166,01611190000000,D,NaN,Alameda Unified,Alameda,NaN,NaN,NaN,asian,1287,NaN,43.6,1336.0,NaN,44.8,-1.2,5,3,5,30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018,61119
167,01611190000000,D,NaN,Alameda Unified,Alameda,NaN,NaN,NaN,english_learners,1081,NaN,-13.7,1177.0,NaN,-12.9,-0.9,3,3,3,130,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018,61119


In [32]:
#creating district-level math performance variable (dictionary)
dfm = math
dfm = pandas.concat([dfm.pop(x) for x in ['District Code',
                                          'districtname',                                          
                                          'studentgroup', 
                                          'currstatus', 
                                          'priorstatus', 
                                          'change']],
                    1)
# slice out columns

In [33]:
dfm.shape

(13603, 6)

In [34]:
dfm

,District Code,districtname,studentgroup,currstatus,priorstatus,change
163,61119,Alameda Unified,african_american,-83.9,-82.8,-1.1
164,61119,Alameda Unified,american_indian,-51.8,-50.0,-1.8
165,61119,Alameda Unified,all_students,14.3,10.8,3.4
166,61119,Alameda Unified,asian,43.6,44.8,-1.2
167,61119,Alameda Unified,english_learners,-13.7,-12.9,-0.9
...,...,...,...,...,...,...
148683,72769,Wheatland Union High,pacific_islander,NaN,NaN,NaN
148684,72769,Wheatland Union High,rfep_only,NaN,NaN,NaN
148685,72769,Wheatland Union High,ses_disad,-90.0,-74.0,-16.0
148686,72769,Wheatland Union High,disabilities,-226.6,NaN,NaN


# LCFF funding data

[FAQ here](https://www.cde.ca.gov/fg/aa/lc/lcfffaq.asp#CALPADS)

In [35]:
lcff = pandas.read_csv('lcff_data/lcffsnapshot18an.csv')
lcff.shape

(2243, 26)

In [36]:
lcff.columns
# leading AND trailing spaces! inconsistently!! 

Index([' County Code ', 'District Code', 'School Code',
       'Local Educational Agency ', 'Charter Number',
       ' TK/K-3 ADA\nTarget, B-5/B-1 ', ' 4 - 6 ADA\nTarget, B-6/B-2 ',
       ' 7 - 8 ADA\nTarget, B-7/B-3 ', ' 9 - 12 ADA\nTarget, B-8/B-4 ',
       'Unduplicated Pupil Percentage\nTarget, D-1',
       ' Base Grant Funding\nTarget, C-5 ',
       ' Supplemental Grant Funding\nTarget, D-7 ',
       ' Concentration Grant Funding\nTarget, E-8 ',
       ' Necessary Small Schools Allowance\nTarget, F-1 ',
       ' Add-On (Based on 2012-13 Targeted Instructional Improvement Block Grant)\nTarget, F-2 ',
       ' Add-On (Based on 2012-13 Home-to-School Transportation)\nTarget, F-3 ',
       ' Add-On (Based on 2012-13 Small School District Bus Replacement Program)\nTarget, F-4 ',
       ' Total LCFF Target Entitlement\nTarget, G-1/F-1 ',
       ' Total LCFF Floor Entitlement\nTransition, B-13 ',
       ' Current Year Gap Funding (100%)\nTransition, C-3 ',
       ' Economic Recovery Tar

In [37]:
lcff.columns = lcff.columns.str.strip()
# strip leading/trailing spaces

In [38]:
lcff_state_totals = lcff.tail(1)
lcff.drop(lcff.tail(1).index,inplace=True)
# extracting state totals, removing from df

In [39]:
# Slicing by district level cds code
lcff['County Code'] = lcff['County Code'].astype(str)
lcff['District Code'] = lcff['District Code'].astype(int)
lcff['District Code'] = lcff['District Code'].astype(str)
lcff['School Code'] = lcff['School Code'].astype(int)
lcff['School Code'] = lcff['School Code'].astype(str)
# Changing to strings
lcff['School Code'] = lcff['School Code'].apply(lambda x: '{0:0>7}'.format(x))
# Front fill school codes to == 7
lcff = lcff.loc[lcff['School Code'] == '0000000']
# select districts, district entries do not have a school level cds code
lcff['cds'] = lcff['County Code'] + lcff['District Code'] + lcff['School Code']
# Combining into cds code
col_name='cds'
first_col = lcff.pop(col_name)
lcff.insert(0, col_name, first_col)
# Moving to front 
lcff.drop('County Code', axis=1, inplace=True)
# lcff.drop('District Code', axis=1, inplace=True)
lcff.drop('School Code', axis=1, inplace=True)
len(lcff)

944

In [40]:
# renaming important columns
lcff = lcff.rename(columns={'Charter Number': 'charter_number',
                     'Unduplicated Pupil Percentage\nTarget, D-1': 'unduplicated_pupil_percentage',
                     'Base Grant Funding\nTarget, C-5': 'base_grant',
                     'Supplemental Grant Funding\nTarget, D-7': 'supplemental_grant',
                     'Concentration Grant Funding\nTarget, E-8': 'concentration_grant',
                     'Total LCFF Target Entitlement\nTarget, G-1/F-1': 'total_grants',
                     'Unduplicated Pupil Percentage\nTarget, D-1': 'unduplicated_pupil_count',
                     'Necessary Small Schools Allowance\nTarget, F-1': 'nec_small_schools'
                           })

In [41]:
lcff.head()
# cds codes are now concurrent with documentation and comparable to acheivement data

,cds,District Code,Local Educational Agency,charter_number,"TK/K-3 ADA\nTarget, B-5/B-1","4 - 6 ADA\nTarget, B-6/B-2","7 - 8 ADA\nTarget, B-7/B-3","9 - 12 ADA\nTarget, B-8/B-4",unduplicated_pupil_count,base_grant,supplemental_grant,concentration_grant,nec_small_schools,"Add-On (Based on 2012-13 Targeted Instructional Improvement Block Grant)\nTarget, F-2","Add-On (Based on 2012-13 Home-to-School Transportation)\nTarget, F-3","Add-On (Based on 2012-13 Small School District Bus Replacement Program)\nTarget, F-4",total_grants,"Total LCFF Floor Entitlement\nTransition, B-13","Current Year Gap Funding (100%)\nTransition, C-3","Economic Recovery Target\nTransition, D-1","Miscellaneous Adjustments\nTransition, E-1","Total Local Revenue or In-Lieu of Property Taxes\nTransition, F-2/F-7","Education Protection Account Entitlement\nTransition, F-5/F-9","Net State Aid\nTransition, F-6/F-10","Additional SA for MSA Guarantee\nTransition, H-1/H-2"
10,01611190000000,61119,Alameda Unified,NaN,"2,870.43","1,917.91","1,219.66","3,050.17",0.3486,"75,938,983","5,294,466",-,-,"665,778","258,889",-,"82,158,116","77,176,229","4,981,887",-,-,"29,277,891","14,772,333","38,107,892",-
16,01611270000000,61127,Albany City Unified,NaN,"1,060.09",806.37,554.25,"1,131.64",0.2833,"29,644,972","1,679,684",-,-,-,"27,884",-,"31,352,540","29,460,891","1,891,649",-,-,"11,005,115","5,777,202","14,570,223",-
17,01611430000000,61143,Berkeley Unified,NaN,"2,828.54","2,101.08","1,397.55","3,070.55",0.3519,"78,556,532","5,528,808",-,-,"4,293,233","991,048",-,"89,369,621","84,726,528","4,643,093","820,253",-,"43,425,650","8,875,387","37,888,837",-
19,01611500000000,61150,Castro Valley Unified,NaN,"2,617.44","2,032.50","1,400.40","2,918.48",0.2904,"74,911,585","4,350,864",-,-,-,"292,158",-,"79,554,607","74,489,641","5,064,966",-,-,"26,572,525","14,660,230","38,321,852",-
20,01611680000000,61168,Emery Unified,NaN,237.65,151.54,104.99,191.97,0.7953,"5,702,229","906,997","699,378",-,-,"18,271",-,"7,326,875","6,698,947","627,928","108,723",-,"5,582,328","137,230","1,716,040",-


In [42]:
#is this done?
dflcff = lcff[['District Code', 'Local Educational Agency', 'unduplicated_pupil_count', 'base_grant', 'supplemental_grant', 'concentration_grant', 'total_grants', 'cds' ]].copy()
dflcff.head()

,District Code,Local Educational Agency,unduplicated_pupil_count,base_grant,supplemental_grant,concentration_grant,total_grants,cds
10,61119,Alameda Unified,0.3486,"75,938,983","5,294,466",-,"82,158,116",01611190000000
16,61127,Albany City Unified,0.2833,"29,644,972","1,679,684",-,"31,352,540",01611270000000
17,61143,Berkeley Unified,0.3519,"78,556,532","5,528,808",-,"89,369,621",01611430000000
19,61150,Castro Valley Unified,0.2904,"74,911,585","4,350,864",-,"79,554,607",01611500000000
20,61168,Emery Unified,0.7953,"5,702,229","906,997","699,378","7,326,875",01611680000000


# Free and reduced meal plan data
Used by policy makers as a stand-in for a  segregation indicator. 

Find the codebook [here](https://www.cde.ca.gov/ds/sd/sd/fsspfrpm.asp).

In [113]:
frpm = pandas.read_csv('lcff_data/frpm1819.csv', encoding = "ISO-8859-1", skiprows=1)
frpm.shape
# aggregate figures by district
# jump down to 'mealagg' cell

(10520, 28)

In [114]:
# renaming important columns
frpm = frpm.rename(columns={'Percent (%) \nEligible FRPM \n(K-12)': 'percent_eligible_k12_frpm', 
                            'Percent (%) \nEligible Free \n(K-12)': 'percent_eligible_k12_free', 
                            'Percent (%) \nEligible FRPM \n(Ages 5-17)' : 'percent_eligible_by_age',
                            'Enrollment \n(Ages 5-17)': 'enrollment_by_age',
                            'FRPM Count \n(Ages 5-17)': 'frpm_count_by_age',
                            'Free Meal \nCount \n(Ages 5-17)': 'free_count_by_age',
                            'District Type': 'district_type',
                            'District Name': 'district_name',
                            'County Name': 'county_name',
                            
                           })

In [115]:
frpm.head()

,Academic Year,County Code,District Code,School Code,county_name,district_name,School Name,district_type,School Type,Educational \nOption Type,NSLP \nProvision \nStatus,Charter \nSchool \n(Y/N),Charter \nSchool \nNumber,Charter \nFunding \nType,IRC,Low Grade,High Grade,Enrollment \n(K-12),Free Meal \nCount \n(K-12),percent_eligible_k12_free,FRPM Count \n(K-12),percent_eligible_k12_frpm,enrollment_by_age,free_count_by_age,Percent (%) \nEligible Free \n(Ages 5-17),frpm_count_by_age,percent_eligible_by_age,CALPADS Fall 1 \nCertification Status
0,2018-2019,1,10017,112607,Alameda,Alameda County Office of Education,Envision Academy for Arts & Technology,County Office of Education (COE),High Schools (Public),Traditional,NaN,Yes,0811,Directly funded,Y,9,12,385,262,68.1%,306,79.5%,370,249,67.3%,291,78.6%,Y
1,2018-2019,1,10017,123968,Alameda,Alameda County Office of Education,Community School for Creative Education,County Office of Education (COE),Elementary Schools (Public),Traditional,NaN,Yes,1284,Directly funded,Y,K,8,241,118,49.0%,120,49.8%,224,113,50.4%,115,51.3%,Y
2,2018-2019,1,10017,124172,Alameda,Alameda County Office of Education,Yu Ming Charter,County Office of Education (COE),Elementary Schools (Public),Traditional,NaN,Yes,1296,Directly funded,Y,K,8,445,58,13.0%,64,14.4%,445,58,13.0%,64,14.4%,Y
3,2018-2019,1,10017,125567,Alameda,Alameda County Office of Education,Urban Montessori Charter,County Office of Education (COE),Elementary Schools (Public),Traditional,NaN,Yes,1383,Directly funded,Y,K,8,432,113,26.2%,131,30.3%,418,111,26.6%,129,30.9%,Y
4,2018-2019,1,10017,130401,Alameda,Alameda County Office of Education,Alameda County Juvenile Hall/Court,County Office of Education (COE),Juvenile Court Schools,Juvenile Court School,NaN,No,NaN,NaN,N,K,12,53,53,100.0%,53,100.0%,50,50,100.0%,50,100.0%,Y


In [116]:
#creating CDS code
frpm['County Code'] = frpm['County Code'].astype(str)
frpm['County Code'] = frpm['County Code'].apply(lambda x: '{0:0>2}'.format(x))
frpm['District Code'] = frpm['District Code'].astype(str)
frpm['School Code'] = frpm['School Code'].astype(str)
frpm['School Code'] = frpm['School Code'].apply(lambda x: '{0:0>7}'.format(x))
# converting to strings, adding leading zeroes 
frpm['cds'] = frpm['County Code'] + frpm['District Code'] + frpm['School Code']
# combining codes

frpm = frpm[frpm.district_type !='County Office of Education (COE)']
# removing county offices of education

col_name1='cds'
first_col1 = frpm.pop(col_name1)
frpm.insert(0, col_name1, first_col1)
# Moving to front

frpm.drop('County Code', axis=1, inplace=True)
# frpm.drop('District Code', axis=1, inplace=True)
frpm.drop('School Code', axis=1, inplace=True)
# dropping split codes

In [117]:
frpm.columns

Index(['cds', 'Academic Year', 'District Code', 'county_name', 'district_name',
       'School Name', 'district_type', 'School Type',
       'Educational \nOption Type', 'NSLP \nProvision \nStatus',
       'Charter \nSchool \n(Y/N)', 'Charter \nSchool \nNumber',
       'Charter \nFunding \nType', 'IRC', 'Low Grade', 'High Grade',
       'Enrollment \n(K-12)', 'Free Meal \nCount \n(K-12)',
       'percent_eligible_k12_free', 'FRPM Count \n(K-12)',
       'percent_eligible_k12_frpm', 'enrollment_by_age', 'free_count_by_age',
       'Percent (%) \nEligible Free \n(Ages 5-17)', 'frpm_count_by_age',
       'percent_eligible_by_age', 'CALPADS Fall 1 \nCertification Status'],
      dtype='object')

In [118]:
# aggregate all schools into district level

frpm['enrollment_by_age'] = frpm['enrollment_by_age'].str.replace(',', '').astype(int)
frpm['frpm_count_by_age'] = frpm['frpm_count_by_age'].str.replace(',', '').astype(int)
frpm['free_count_by_age'] = frpm['free_count_by_age'].str.replace(',', '').astype(int)
# striping commas, converting to int 

In [119]:
frpm.columns

Index(['cds', 'Academic Year', 'District Code', 'county_name', 'district_name',
       'School Name', 'district_type', 'School Type',
       'Educational \nOption Type', 'NSLP \nProvision \nStatus',
       'Charter \nSchool \n(Y/N)', 'Charter \nSchool \nNumber',
       'Charter \nFunding \nType', 'IRC', 'Low Grade', 'High Grade',
       'Enrollment \n(K-12)', 'Free Meal \nCount \n(K-12)',
       'percent_eligible_k12_free', 'FRPM Count \n(K-12)',
       'percent_eligible_k12_frpm', 'enrollment_by_age', 'free_count_by_age',
       'Percent (%) \nEligible Free \n(Ages 5-17)', 'frpm_count_by_age',
       'percent_eligible_by_age', 'CALPADS Fall 1 \nCertification Status'],
      dtype='object')

In [120]:
names = frpm['district_name'].unique()
len(names)

966

In [174]:
w1 = dict(zip(frpm.district_name,frpm.county_name))
w1

{'California School for the Blind (State Special Schl)': 'Alameda',
 'California School for the Deaf-Fremont (State Special Schl)': 'Alameda',
 'Alameda Unified': 'Alameda',
 'Albany City Unified': 'Alameda',
 'Berkeley Unified': 'Alameda',
 'Castro Valley Unified': 'Alameda',
 'Emery Unified': 'Alameda',
 'Fremont Unified': 'Alameda',
 'Hayward Unified': 'Alameda',
 'Livermore Valley Joint Unified': 'Alameda',
 'Mountain House Elementary': 'Alameda',
 'Newark Unified': 'Alameda',
 'New Haven Unified': 'Alameda',
 'Oakland Unified': 'Alameda',
 'Piedmont City Unified': 'Alameda',
 'San Leandro Unified': 'Alameda',
 'San Lorenzo Unified': 'Alameda',
 'Dublin Unified': 'Alameda',
 'Pleasanton Unified': 'Alameda',
 'Sunol Glen Unified': 'Alameda',
 'SBE - Latitude 37.8 High': 'Alameda',
 'Alpine County Unified': 'Alpine',
 'Amador County Unified': 'Amador',
 'Bangor Union Elementary': 'Butte',
 'Biggs Unified': 'Butte',
 'Chico Unified': 'Butte',
 'Durham Unified': 'Butte',
 'Feather Fall

In [191]:
s = pandas.Series(w1, name='county_name')
s = s.to_frame()
s.index.name = 'district_name'

In [192]:
s.index.name = 'district_name'

In [193]:
s.reset_index()

,district_name,county_name
0,California School for the Blind (State Special...,Alameda
1,California School for the Deaf-Fremont (State ...,Alameda
2,Alameda Unified,Alameda
3,Albany City Unified,Alameda
4,Berkeley Unified,Alameda
...,...,...
961,Camptonville Elementary,Yuba
962,Marysville Joint Unified,Yuba
963,Plumas Lake Elementary,Yuba
964,Wheatland,Yuba


In [163]:
codes = []
for name in names:
    temp = frpm[frpm['district_name'] == name]
    codes.append(temp['District Code'].values[0])

In [161]:
counties = frpm['county_name'].unique()
len(counties)

58

In [123]:
countnames = []
for county in counties:
        temp2 = frpm[frpm['county_name'] == county]
        countnames.append(temp2['District Code'].values[0])

In [165]:
t6 = pandas.DataFrame(list(zip(counties, countnames)),
                     columns =['county_name', 'fips',])

t6

,county_name,fips
0,Alameda,31609
1,Alpine,61333
2,Amador,73981
3,Butte,61382
4,Calaveras,61556
5,Colusa,61598
6,Contra Costa,61630
7,Del Norte,61820
8,El Dorado,61838
9,Fresno,61994


In [175]:
t5 = pandas.DataFrame(list(zip(names, codes,)), 
               columns =['district_name', 'District Code',])
t5

,district_name,District Code
0,California School for the Blind (State Special...,31609
1,California School for the Deaf-Fremont (State ...,31609
2,Alameda Unified,61333
3,Albany City Unified,73981
4,Berkeley Unified,61382
...,...,...
961,Camptonville Elementary,64014
962,Marysville Joint Unified,64089
963,Plumas Lake Elementary,64212
964,Wheatland,65177


In [194]:
s.head()

,county_name
district_name,
California School for the Blind (State Special Schl),Alameda
California School for the Deaf-Fremont (State Special Schl),Alameda
Alameda Unified,Alameda
Albany City Unified,Alameda
Berkeley Unified,Alameda


In [129]:
t6 = t5.merge(cfips, left_on='lkey', right_on='rkey')

,district_name,District Code
0,California School for the Blind (State Special...,31609
1,California School for the Deaf-Fremont (State ...,31617
2,Alameda Unified,61119
3,Albany City Unified,61127
4,Berkeley Unified,61143
...,...,...
961,Camptonville Elementary,72728
962,Marysville Joint Unified,72736
963,Plumas Lake Elementary,72744
964,Wheatland,72751


In [55]:
# If nothing is int (and everything is object), all columns live through the groupby
# if we convert to int before groupby (to make .sum() work), only int lives through groupby

dffrpm = frpm.groupby(by='district_name').sum()
# aggregating

In [56]:
dffrpm.columns

Index(['enrollment_by_age', 'free_count_by_age', 'frpm_count_by_age'], dtype='object')

In [57]:
dffrpm['pct_frpm_by_age'] = dffrpm.frpm_count_by_age / dffrpm.enrollment_by_age
dffrpm['pct_free_by_age'] = dffrpm.free_count_by_age / dffrpm.enrollment_by_age
# inferring more convinient than copying lul

In [58]:
dffrpm.shape
# need to include district name and county name in the aggreagtion
# why do we lose columns in the groupby operation? dtypes object preventing groupby?

(966, 5)

In [59]:
t5.shape

(966, 2)

In [195]:
t6 = pandas.merge(dffrpm, t5, how='left', on=['district_name'])
t6

,district_name,enrollment_by_age,free_count_by_age,frpm_count_by_age,pct_frpm_by_age,pct_free_by_age,District Code_x,District Code_y
0,ABC Unified,20022,8508,9994,0.499151,0.424933,64212,62554
1,Acalanes Union High,5487,334,358,0.065245,0.060871,61630,69856
2,Ackerman Charter,584,154,180,0.308219,0.263699,66761,71662
3,Acton-Agua Dulce Unified,13553,5745,6389,0.471409,0.423891,75309,65961
4,Adelanto Elementary,8512,6158,6708,0.788064,0.723449,67587,61333
...,...,...,...,...,...,...,...,...
961,Yosemite Unified,2036,881,977,0.479862,0.432711,76414,70185
962,Yreka Union Elementary,984,634,702,0.713415,0.644309,70508,61994
963,Yreka Union High,610,269,290,0.475410,0.440984,70516,62554
964,Yuba City Unified,12839,8182,9518,0.741335,0.637277,71464,67967


In [204]:
t7 = pandas.merge(s, t6, how='left', on=['district_name'])
t7.drop('District Code_y', axis=1, inplace=True)
t7 = t7.rename(columns={'District Code_x': 'District Code',})
t7

,district_name,county_name,enrollment_by_age,free_count_by_age,frpm_count_by_age,pct_frpm_by_age,pct_free_by_age,District Code
0,California School for the Blind (State Special...,Alameda,42,42,42,1.000000,1.000000,31609
1,California School for the Deaf-Fremont (State ...,Alameda,311,311,311,1.000000,1.000000,31617
2,Alameda Unified,Alameda,11031,2953,3306,0.299701,0.267700,61119
3,Albany City Unified,Alameda,3612,647,742,0.205426,0.179125,61127
4,Berkeley Unified,Alameda,10022,2758,3153,0.314608,0.275195,61143
...,...,...,...,...,...,...,...,...
961,Camptonville Elementary,Yuba,475,200,260,0.547368,0.421053,72728
962,Marysville Joint Unified,Yuba,9963,7226,7833,0.786209,0.725284,72736
963,Plumas Lake Elementary,Yuba,1301,374,478,0.367410,0.287471,72744
964,Wheatland,Yuba,1364,457,644,0.472141,0.335044,72751


In [214]:
cfips['county_name'] = cfips['county_name'].str.replace('County', '')
cfips

,county_name,GEOID
0,Alameda,06001
1,Alpine,06003
2,Amador,06005
3,Butte,06007
4,Calaveras,06009
5,Colusa,06011
6,Contra Costa,06013
7,Del Norte,06015
8,El Dorado,06017
9,Fresno,06019


In [247]:
t7.county_name = t7.county_name.str.strip()
cfips.county_name = cfips.county_name.str.strip()
adistricts.NAME = adistricts.NAME.str.strip()

In [239]:
t8 = t7.merge(cfips, how='left', on=['county_name'])
t8

,district_name,county_name,enrollment_by_age,free_count_by_age,frpm_count_by_age,pct_frpm_by_age,pct_free_by_age,District Code,GEOID
0,California School for the Blind (State Special...,Alameda,42,42,42,1.000000,1.000000,31609,06001
1,California School for the Deaf-Fremont (State ...,Alameda,311,311,311,1.000000,1.000000,31617,06001
2,Alameda Unified,Alameda,11031,2953,3306,0.299701,0.267700,61119,06001
3,Albany City Unified,Alameda,3612,647,742,0.205426,0.179125,61127,06001
4,Berkeley Unified,Alameda,10022,2758,3153,0.314608,0.275195,61143,06001
...,...,...,...,...,...,...,...,...,...
961,Camptonville Elementary,Yuba,475,200,260,0.547368,0.421053,72728,06115
962,Marysville Joint Unified,Yuba,9963,7226,7833,0.786209,0.725284,72736,06115
963,Plumas Lake Elementary,Yuba,1301,374,478,0.367410,0.287471,72744,06115
964,Wheatland,Yuba,1364,457,644,0.472141,0.335044,72751,06115


In [244]:
adistricts['NAME'] = adistricts['NAME'].str.replace('School District', '')

In [251]:
adistricts.shape

(986, 18)

In [248]:
g1 = t8.merge(adistricts, how='left', left_on='district_name', right_on='NAME')
g1
# deal with the duplicated names (pioneer, jefferson, lakeside)

,district_name,county_name,enrollment_by_age,free_count_by_age,frpm_count_by_age,pct_frpm_by_age,pct_free_by_age,District Code,GEOID_x,STATEFP,UNSDLEA,GEOID_y,NAME,LSAD,LOGRADE,HIGRADE,MTFCC,SDTYP,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry,origin,ELSDLEA,SCSDLEA
0,California School for the Blind (State Special...,Alameda,42,42,42,1.000000,1.000000,31609,06001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN
1,California School for the Deaf-Fremont (State ...,Alameda,311,311,311,1.000000,1.000000,31617,06001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN
2,Alameda Unified,Alameda,11031,2953,3306,0.299701,0.267700,61119,06001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN
3,Albany City Unified,Alameda,3612,647,742,0.205426,0.179125,61127,06001,06,01860,06018,Albany City Unified,00,KG,12,G5420,None,E,4.634305e+06,9523833.0,+37.8906499,-122.3181164,"POLYGON ((-122.37312 37.88388, -122.37114 37.8...",unified,NaN,NaN
4,Berkeley Unified,Alameda,10022,2758,3153,0.314608,0.275195,61143,06001,06,04740,06047,Berkeley Unified,00,KG,12,G5420,None,E,2.709528e+07,18715614.0,+37.8669717,-122.2991474,"POLYGON ((-122.36778 37.86673, -122.35671 37.8...",unified,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
975,Camptonville Elementary,Yuba,475,200,260,0.547368,0.421053,72728,06115,06,NaN,06072,Camptonville Elementary,00,KG,12,G5400,None,E,1.356720e+08,10983925.0,+39.4308753,-121.1005893,"POLYGON ((-121.16298 39.47496, -121.16254 39.4...",elementary,07260,NaN
976,Marysville Joint Unified,Yuba,9963,7226,7833,0.786209,0.725284,72736,06115,06,24090,06240,Marysville Joint Unified,00,KG,12,G5420,None,E,1.196179e+09,18292938.0,+39.3278994,-121.3543265,"POLYGON ((-121.63637 39.24608, -121.63629 39.2...",unified,NaN,NaN
977,Plumas Lake Elementary,Yuba,1301,374,478,0.367410,0.287471,72744,06115,06,NaN,06311,Plumas Lake Elementary,00,KG,08,G5400,None,E,9.471641e+07,926165.0,+39.0201102,-121.5317817,"POLYGON ((-121.58904 38.99152, -121.58838 38.9...",elementary,31180,NaN
978,Wheatland,Yuba,1364,457,644,0.472141,0.335044,72751,06115,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN


## Test Variables for merge with District Codes

In [61]:
geot = set(adsgeo)
len(geot)
# There are no district codes in the geometries - must match by district name. 

970

In [62]:
frpmt = set(dffrpm['District Code'])
len(frpmt)

966

In [63]:
lcfft = set(dflcff['District Code'])
len(lcfft)

944

In [64]:
elat = set(dfe['District Code'])
len(elat)

932

In [65]:
matht = set(dfm['District Code'])
len(matht)
# lowest common denominator

931

In [66]:
elat.difference(matht)

{'70318'}

In [67]:
d2 = lcfft.difference(matht)
d2

{'61994',
 '63933',
 '66761',
 '68627',
 '70318',
 '70680',
 '70714',
 '70730',
 '70797',
 '70839',
 '70854',
 '70862',
 '70912',
 '70920',
 '71274',
 '71670'}

In [68]:
d3 = lcfft.difference(elat)
d3

{'61994',
 '63933',
 '66761',
 '68627',
 '70680',
 '70714',
 '70730',
 '70797',
 '70839',
 '70854',
 '70862',
 '70912',
 '70920',
 '71274',
 '71670'}

In [69]:
drops = d2.intersection(d3)
len(drops)

15

In [70]:
d2.difference(drops)

{'70318'}

In [71]:
drops = d2
len(drops)

16

In [72]:
d4 = frpmt.difference(matht)
len(d4)

49

In [73]:
d2.difference(d4)

set()

In [74]:
d3.difference(d4)

set()

### d4 contains the list of districts to drop to merge all 4 datasets. 

In [75]:
len(d4)

49

In [76]:
d4

{'31609',
 '31617',
 '31625',
 '32276',
 '61994',
 '63933',
 '66761',
 '68627',
 '70318',
 '70680',
 '70714',
 '70730',
 '70797',
 '70839',
 '70854',
 '70862',
 '70912',
 '70920',
 '71274',
 '71670',
 '75630',
 '75663',
 '75697',
 '76547',
 '76885',
 '76893',
 '76901',
 '76919',
 '76927',
 '76943',
 '76950',
 '76968',
 '76992',
 '77024',
 '77032',
 '77065',
 '77081',
 '77099',
 '77107',
 '77131',
 '77149',
 '77156',
 '77164',
 '77172',
 '77180',
 '77198',
 '77206',
 '77214',
 '77222'}

# Examining data

### State-level performance data

In [77]:
ela_state

,studentgroup,currstatus,priorstatus,change,currdenom
3,asian,62.4,60.5,2.0,293002
7,filipino,44.0,41.2,2.7,71139
11,multiple_races,28.6,26.3,2.3,118753
16,white,27.7,26.9,0.8,734863
6,english_only,6.5,4.6,1.8,1795002
13,rfep_only,2.2,-0.4,2.7,488023
2,all_students,-6.0,-8.2,2.2,3171704
12,pacific_islander,-21.3,-22.0,0.7,14894
9,hispanic,-31.3,-34.5,3.2,1738615
14,ses_disad,-34.7,-38.7,4.0,1982522


how to loop?

```for score in student group
    select student group, return averaged score```

In [78]:
# both english learning groups
(-47.1 + -94.1) / 2

-70.6

In [79]:
# combine `studentgroups`
# lcff_group = 'ses_disad', 'english_learners', 'homeless_youth', 'foster_youth' 'english_learners_only'

d = (-34.7 + -47.1 + -51.0 + -79.2 + -94.1)
d / 5

-61.220000000000006

In [80]:
math_state

,studentgroup,currstatus,priorstatus,change,currdenom
3,asian,56.7,54.3,2.4,293068
7,filipino,13.1,10.3,2.8,71096
11,multiple_races,1.9,0.1,1.9,118473
16,white,-1.0,-1.6,0.6,733213
6,english_only,-25.2,-26.3,1.1,1790583
13,rfep_only,-29.7,-31.1,1.4,487555
2,all_students,-36.4,-37.7,1.3,3166345
12,pacific_islander,-52.0,-51.3,-0.7,14849
9,hispanic,-65.8,-67.5,1.7,1735982
14,ses_disad,-67.4,-69.7,2.3,1978909


In [81]:
dfe = dfe.rename(columns={'currstatus': 'ela_currstatus',
                          'priorstatus': 'ela_priorstatus',
                          'change' : 'ela_change'
                         })
dfe
# dataframe english

,District Code,districtname,studentgroup,ela_currstatus,ela_priorstatus,ela_change
164,61119,Alameda Unified,african_american,-43.0,-48.7,5.7
165,61119,Alameda Unified,american_indian,-14.8,14.3,-29.1
166,61119,Alameda Unified,all_students,36.8,32.8,4.0
167,61119,Alameda Unified,asian,48.6,54.0,-5.3
168,61119,Alameda Unified,english_learners,-2.5,-1.4,-1.1
...,...,...,...,...,...,...
148816,72769,Wheatland Union High,pacific_islander,NaN,NaN,NaN
148817,72769,Wheatland Union High,rfep_only,NaN,NaN,NaN
148818,72769,Wheatland Union High,ses_disad,-28.4,-7.9,-20.6
148819,72769,Wheatland Union High,disabilities,-150.3,NaN,NaN


In [82]:
dfm = dfm.rename(columns={'currstatus': 'm_currstatus',
                          'priorstatus': 'm_priorstatus',
                          'change' : 'm_change'
                         })
dfm
# dataframe math

,District Code,districtname,studentgroup,m_currstatus,m_priorstatus,m_change
163,61119,Alameda Unified,african_american,-83.9,-82.8,-1.1
164,61119,Alameda Unified,american_indian,-51.8,-50.0,-1.8
165,61119,Alameda Unified,all_students,14.3,10.8,3.4
166,61119,Alameda Unified,asian,43.6,44.8,-1.2
167,61119,Alameda Unified,english_learners,-13.7,-12.9,-0.9
...,...,...,...,...,...,...
148683,72769,Wheatland Union High,pacific_islander,NaN,NaN,NaN
148684,72769,Wheatland Union High,rfep_only,NaN,NaN,NaN
148685,72769,Wheatland Union High,ses_disad,-90.0,-74.0,-16.0
148686,72769,Wheatland Union High,disabilities,-226.6,NaN,NaN


In [83]:
dflcff
# dataframe lcff

,District Code,Local Educational Agency,unduplicated_pupil_count,base_grant,supplemental_grant,concentration_grant,total_grants,cds
10,61119,Alameda Unified,0.3486,"75,938,983","5,294,466",-,"82,158,116",01611190000000
16,61127,Albany City Unified,0.2833,"29,644,972","1,679,684",-,"31,352,540",01611270000000
17,61143,Berkeley Unified,0.3519,"78,556,532","5,528,808",-,"89,369,621",01611430000000
19,61150,Castro Valley Unified,0.2904,"74,911,585","4,350,864",-,"79,554,607",01611500000000
20,61168,Emery Unified,0.7953,"5,702,229","906,997","699,378","7,326,875",01611680000000
...,...,...,...,...,...,...,...,...
2233,72728,Camptonville Elementary,0.6988,"420,692","58,796","31,299","537,244",58727280000000
2235,72736,Marysville Joint Unified,0.8168,"76,193,862","12,447,028","10,164,261","100,261,913",58727360000000
2238,72744,Plumas Lake Elementary,0.3868,"10,261,440","793,825",-,"11,108,901",58727440000000
2239,72751,Wheatland,0.4783,"9,947,133","951,543",-,"11,234,349",58727510000000


In [84]:
dffrpm

,district_name,enrollment_by_age,free_count_by_age,frpm_count_by_age,pct_frpm_by_age,pct_free_by_age,District Code
0,ABC Unified,20022,8508,9994,0.499151,0.424933,64212
1,Acalanes Union High,5487,334,358,0.065245,0.060871,61630
2,Ackerman Charter,584,154,180,0.308219,0.263699,66761
3,Acton-Agua Dulce Unified,13553,5745,6389,0.471409,0.423891,75309
4,Adelanto Elementary,8512,6158,6708,0.788064,0.723449,67587
...,...,...,...,...,...,...,...
961,Yosemite Unified,2036,881,977,0.479862,0.432711,76414
962,Yreka Union Elementary,984,634,702,0.713415,0.644309,70508
963,Yreka Union High,610,269,290,0.475410,0.440984,70516
964,Yuba City Unified,12839,8182,9518,0.741335,0.637277,71464


### Merging lcff and frpm (done)

In [85]:
q1 = dflcff['District Code'].values
len(q1)

944

In [86]:
q2 = dffrpm['District Code'].values
len(q2)

966

In [87]:
s1 = set(dflcff['District Code'])
len(s1)

944

In [88]:
s2 = set(dffrpm['District Code'])

In [89]:
s2

{'31609',
 '31617',
 '31625',
 '32276',
 '61119',
 '61127',
 '61143',
 '61150',
 '61168',
 '61176',
 '61192',
 '61200',
 '61218',
 '61234',
 '61242',
 '61259',
 '61275',
 '61291',
 '61309',
 '61333',
 '61382',
 '61408',
 '61424',
 '61432',
 '61440',
 '61457',
 '61499',
 '61507',
 '61515',
 '61523',
 '61531',
 '61549',
 '61556',
 '61564',
 '61572',
 '61580',
 '61598',
 '61606',
 '61614',
 '61622',
 '61630',
 '61648',
 '61655',
 '61663',
 '61671',
 '61697',
 '61705',
 '61713',
 '61721',
 '61739',
 '61747',
 '61754',
 '61762',
 '61770',
 '61788',
 '61796',
 '61804',
 '61812',
 '61820',
 '61838',
 '61846',
 '61853',
 '61879',
 '61887',
 '61895',
 '61903',
 '61911',
 '61929',
 '61952',
 '61960',
 '61978',
 '61986',
 '61994',
 '62026',
 '62042',
 '62109',
 '62117',
 '62125',
 '62158',
 '62166',
 '62240',
 '62257',
 '62265',
 '62281',
 '62323',
 '62331',
 '62356',
 '62364',
 '62372',
 '62380',
 '62414',
 '62430',
 '62513',
 '62539',
 '62547',
 '62554',
 '62596',
 '62638',
 '62646',
 '62653',


In [90]:
s3 = s2.difference(s1)
len(s3)

34

In [91]:
s4 = s1.difference(s2)
len(s4)

12

In [92]:
f1 = dflcff[~dflcff['District Code'].isin(s4)]
f1

,District Code,Local Educational Agency,unduplicated_pupil_count,base_grant,supplemental_grant,concentration_grant,total_grants,cds
10,61119,Alameda Unified,0.3486,"75,938,983","5,294,466",-,"82,158,116",01611190000000
16,61127,Albany City Unified,0.2833,"29,644,972","1,679,684",-,"31,352,540",01611270000000
17,61143,Berkeley Unified,0.3519,"78,556,532","5,528,808",-,"89,369,621",01611430000000
19,61150,Castro Valley Unified,0.2904,"74,911,585","4,350,864",-,"79,554,607",01611500000000
20,61168,Emery Unified,0.7953,"5,702,229","906,997","699,378","7,326,875",01611680000000
...,...,...,...,...,...,...,...,...
2233,72728,Camptonville Elementary,0.6988,"420,692","58,796","31,299","537,244",58727280000000
2235,72736,Marysville Joint Unified,0.8168,"76,193,862","12,447,028","10,164,261","100,261,913",58727360000000
2238,72744,Plumas Lake Elementary,0.3868,"10,261,440","793,825",-,"11,108,901",58727440000000
2239,72751,Wheatland,0.4783,"9,947,133","951,543",-,"11,234,349",58727510000000


In [93]:
f2 = dffrpm[~dffrpm['District Code'].isin(s3)]
f2

,district_name,enrollment_by_age,free_count_by_age,frpm_count_by_age,pct_frpm_by_age,pct_free_by_age,District Code
0,ABC Unified,20022,8508,9994,0.499151,0.424933,64212
1,Acalanes Union High,5487,334,358,0.065245,0.060871,61630
2,Ackerman Charter,584,154,180,0.308219,0.263699,66761
3,Acton-Agua Dulce Unified,13553,5745,6389,0.471409,0.423891,75309
4,Adelanto Elementary,8512,6158,6708,0.788064,0.723449,67587
...,...,...,...,...,...,...,...
961,Yosemite Unified,2036,881,977,0.479862,0.432711,76414
962,Yreka Union Elementary,984,634,702,0.713415,0.644309,70508
963,Yreka Union High,610,269,290,0.475410,0.440984,70516
964,Yuba City Unified,12839,8182,9518,0.741335,0.637277,71464


In [94]:
# this is done!!!!
df2 = f1.merge(f2, on='District Code')
df2

,District Code,Local Educational Agency,unduplicated_pupil_count,base_grant,supplemental_grant,concentration_grant,total_grants,cds,district_name,enrollment_by_age,free_count_by_age,frpm_count_by_age,pct_frpm_by_age,pct_free_by_age
0,61119,Alameda Unified,0.3486,"75,938,983","5,294,466",-,"82,158,116",01611190000000,Alameda Unified,11031,2953,3306,0.299701,0.267700
1,61127,Albany City Unified,0.2833,"29,644,972","1,679,684",-,"31,352,540",01611270000000,Albany City Unified,3612,647,742,0.205426,0.179125
2,61143,Berkeley Unified,0.3519,"78,556,532","5,528,808",-,"89,369,621",01611430000000,Berkeley Unified,10022,2758,3153,0.314608,0.275195
3,61150,Castro Valley Unified,0.2904,"74,911,585","4,350,864",-,"79,554,607",01611500000000,Castro Valley Unified,9106,1811,2178,0.239183,0.198880
4,61168,Emery Unified,0.7953,"5,702,229","906,997","699,378","7,326,875",01611680000000,Emery Unified,727,515,583,0.801926,0.708391
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
927,72728,Camptonville Elementary,0.6988,"420,692","58,796","31,299","537,244",58727280000000,Camptonville Elementary,475,200,260,0.547368,0.421053
928,72736,Marysville Joint Unified,0.8168,"76,193,862","12,447,028","10,164,261","100,261,913",58727360000000,Marysville Joint Unified,9963,7226,7833,0.786209,0.725284
929,72744,Plumas Lake Elementary,0.3868,"10,261,440","793,825",-,"11,108,901",58727440000000,Plumas Lake Elementary,1301,374,478,0.367410,0.287471
930,72751,Wheatland,0.4783,"9,947,133","951,543",-,"11,234,349",58727510000000,Wheatland,1364,457,644,0.472141,0.335044


# WORKING CELLS: performance

In [150]:
dfe2 = dfe[dfe['District Code'] !='70318']
dfe2
# inherent problem: after dropping extra district, still 5 extra rows?

,District Code,districtname,studentgroup,ela_currstatus,ela_priorstatus,ela_change,dfm
164,61119,Alameda Unified,african_american,-43.0,-48.7,5.7,NaN
165,61119,Alameda Unified,american_indian,-14.8,14.3,-29.1,NaN
166,61119,Alameda Unified,all_students,36.8,32.8,4.0,NaN
167,61119,Alameda Unified,asian,48.6,54.0,-5.3,NaN
168,61119,Alameda Unified,english_learners,-2.5,-1.4,-1.1,NaN
...,...,...,...,...,...,...,...
148816,72769,Wheatland Union High,pacific_islander,NaN,NaN,NaN,NaN
148817,72769,Wheatland Union High,rfep_only,NaN,NaN,NaN,NaN
148818,72769,Wheatland Union High,ses_disad,-28.4,-7.9,-20.6,NaN
148819,72769,Wheatland Union High,disabilities,-150.3,NaN,NaN,NaN


In [151]:
dfm.shape

(13603, 6)

In [90]:
dfe.head(50)

,District Code,districtname,studentgroup,ela_currstatus,ela_priorstatus,ela_change
164,61119,Alameda Unified,african_american,-43.0,-48.7,5.7
165,61119,Alameda Unified,american_indian,-14.8,14.3,-29.1
166,61119,Alameda Unified,all_students,36.8,32.8,4.0
167,61119,Alameda Unified,asian,48.6,54.0,-5.3
168,61119,Alameda Unified,english_learners,-2.5,-1.4,-1.1
169,61119,Alameda Unified,english_learners_only,-42.0,-44.7,2.7
170,61119,Alameda Unified,english_only,43.5,38.3,5.2
171,61119,Alameda Unified,filipino,29.8,16.0,13.9
172,61119,Alameda Unified,foster_youth,NaN,NaN,NaN
173,61119,Alameda Unified,hispanic,5.0,0.9,4.1


In [93]:
dfe2 = dfe
dfe2

,District Code,districtname,studentgroup,ela_currstatus,ela_priorstatus,ela_change
164,61119,Alameda Unified,african_american,-43.0,-48.7,5.7
165,61119,Alameda Unified,american_indian,-14.8,14.3,-29.1
166,61119,Alameda Unified,all_students,36.8,32.8,4.0
167,61119,Alameda Unified,asian,48.6,54.0,-5.3
168,61119,Alameda Unified,english_learners,-2.5,-1.4,-1.1
...,...,...,...,...,...,...
148816,72769,Wheatland Union High,pacific_islander,NaN,NaN,NaN
148817,72769,Wheatland Union High,rfep_only,NaN,NaN,NaN
148818,72769,Wheatland Union High,ses_disad,-28.4,-7.9,-20.6
148819,72769,Wheatland Union High,disabilities,-150.3,NaN,NaN


In [123]:
# Create a variable
mscore = []


for row in dfm['District Code'], dfm['studentgroup']:
    mscore.append(row)


dfe2['m_currstatus'] = mscore

# View the dataframe
dfe2

# cannot add (ignore index=True) to append line

ValueError: Length of values does not match length of index

In [ ]:
pdict = {}

In [183]:
values = dfm.values
keys = mkey
for i in keys:
    for x in values:
        pdict[i] = x
pdict

TypeError: 'Series' objects are mutable, thus they cannot be hashed

In [122]:
# Can I use the codes and groups to merge onto original dfe dataframe?
mdist = dfm['District Code']
mgroup = dfm['studentgroup']
mscore = dfm['m_currstatus']
mprio = dfm['m_priorstatus']
mchange = dfm['m_change']
mscores = list(zip(mdist, mgroup, mscore, mprio, mchange))
mscores

[(-83.9, -82.8, -1.1),
 (-51.8, -50.0, -1.8),
 (14.3, 10.8, 3.4),
 (43.6, 44.8, -1.2),
 (-13.7, -12.9, -0.9),
 (-42.4, -42.9, 0.5),
 (17.8, 13.0, 4.7),
 (-3.5, -12.7, 9.3),
 (nan, nan, nan),
 (-24.4, -26.7, 2.3),
 (-112.4, -92.3, -20.1),
 (28.3, 22.8, 5.6),
 (-77.2, -83.0, 5.9),
 (27.8, 26.5, 1.3),
 (-35.3, -38.8, 3.6),
 (-99.5, -104.5, 5.0),
 (29.8, 26.0, 3.8),
 (-41.4, -23.1, -18.3),
 (nan, nan, nan),
 (41.2, 48.1, -6.9),
 (76.2, 74.0, 2.2),
 (30.7, 29.7, 0.9),
 (0.5, -19.2, 19.8),
 (37.9, 47.6, -9.7),
 (20.2, -14.4, 34.6),
 (nan, nan, nan),
 (-2.7, 3.1, -5.8),
 (nan, nan, nan),
 (32.4, 40.5, -8.1),
 (nan, nan, nan),
 (52.2, 64.4, -12.2),
 (0.5, 9.7, -9.2),
 (-55.7, -43.4, -12.3),
 (46.1, 59.3, -13.2),
 (-86.3, -92.8, 6.5),
 (nan, nan, nan),
 (14.9, 6.1, 8.9),
 (24.9, 19.2, 5.7),
 (-52.9, -53.3, 0.4),
 (-81.6, -69.5, -12.1),
 (24.8, 15.2, 9.5),
 (57.8, 4.4, 53.4),
 (-52.8, nan, nan),
 (-22.6, -32.3, 9.7),
 (-86.2, -98.9, 12.7),
 (33.4, 22.4, 10.9),
 (nan, nan, nan),
 (-2.2, -17.4, 15

In [121]:
edist = dfe['District Code']
egroup = dfe['studentgroup']
escore = dfe['ela_currstatus']
eprio = dfe['ela_priorstatus']
echange = dfe['ela_change']
escores = list(zip(edist, egroup, escore, eprio, echange))
escores

[('61119', 'african_american', -43.0, -48.7, 5.7),
 ('61119', 'american_indian', -14.8, 14.3, -29.1),
 ('61119', 'all_students', 36.8, 32.8, 4.0),
 ('61119', 'asian', 48.6, 54.0, -5.3),
 ('61119', 'english_learners', -2.5, -1.4, -1.1),
 ('61119', 'english_learners_only', -42.0, -44.7, 2.7),
 ('61119', 'english_only', 43.5, 38.3, 5.2),
 ('61119', 'filipino', 29.8, 16.0, 13.9),
 ('61119', 'foster_youth', nan, nan, nan),
 ('61119', 'hispanic', 5.0, 0.9, 4.1),
 ('61119', 'homeless_youth', -75.5, -56.7, -18.7),
 ('61119', 'multiple_races', 46.6, 40.4, 6.2),
 ('61119', 'pacific_islander', -27.3, -49.1, 21.8),
 ('61119', 'rfep_only', 55.0, 55.5, -0.5),
 ('61119', 'ses_disad', -13.1, -20.0, 6.9),
 ('61119', 'disabilities', -73.1, -80.4, 7.3),
 ('61119', 'white', 60.8, 54.1, 6.7),
 ('61127', 'african_american', -9.6, 12.2, -21.8),
 ('61127', 'american_indian', nan, nan, nan),
 ('61127', 'all_students', 57.5, 64.1, -6.6),
 ('61127', 'asian', 73.0, 73.7, -0.8),
 ('61127', 'english_learners', 31.0

In [119]:
result = pandas.merge([escores, mscores], right=edist, sort=False)

TypeError: Can only merge Series or DataFrame objects, a <class 'list'> was passed

In [192]:
scores = []
for row in dfm['m_currstatus']:
    scores.append(ignore_index=True)
    
dfe['m_currstatus'] = scores
dfe

TypeError: append() takes no keyword arguments

In [157]:
test = dfe.append(dfm, ignore_index=True)


,District Code,districtname,studentgroup,ela_currstatus,ela_priorstatus,ela_change,dfm,m_currstatus,m_priorstatus,m_change
27021,73874,Oak Park Unified,english_only,NaN,NaN,NaN,NaN,42.3,36.9,5.5
27022,73874,Oak Park Unified,filipino,NaN,NaN,NaN,NaN,49.4,37.6,11.9
27023,73874,Oak Park Unified,hispanic,NaN,NaN,NaN,NaN,3.7,2.9,0.8
27024,73874,Oak Park Unified,multiple_races,NaN,NaN,NaN,NaN,60.8,62.4,-1.6
27025,73874,Oak Park Unified,pacific_islander,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27026,73874,Oak Park Unified,rfep_only,NaN,NaN,NaN,NaN,62.5,37.6,24.9
27027,73874,Oak Park Unified,ses_disad,NaN,NaN,NaN,NaN,1.4,-6.2,7.6
27028,73874,Oak Park Unified,disabilities,NaN,NaN,NaN,NaN,-52.7,-57.9,5.2
27029,73874,Oak Park Unified,white,NaN,NaN,NaN,NaN,37.4,31.4,6.1
27030,73940,Moorpark Unified,african_american,NaN,NaN,NaN,NaN,-21.5,-24.9,3.5


In [135]:
s3 = dfe.combine_first(dfm)
s3.head(200)

,District Code,dfm,districtname,ela_change,ela_currstatus,ela_priorstatus,m_change,m_currstatus,m_priorstatus,studentgroup
163,61119,NaN,Alameda Unified,NaN,NaN,NaN,-1.1,-83.9,-82.8,african_american
164,61119,NaN,Alameda Unified,5.7,-43.0,-48.7,-1.8,-51.8,-50.0,african_american
165,61119,NaN,Alameda Unified,-29.1,-14.8,14.3,3.4,14.3,10.8,american_indian
166,61119,NaN,Alameda Unified,4.0,36.8,32.8,-1.2,43.6,44.8,all_students
167,61119,NaN,Alameda Unified,-5.3,48.6,54.0,-0.9,-13.7,-12.9,asian
168,61119,NaN,Alameda Unified,-1.1,-2.5,-1.4,0.5,-42.4,-42.9,english_learners
169,61119,NaN,Alameda Unified,2.7,-42.0,-44.7,4.7,17.8,13.0,english_learners_only
170,61119,NaN,Alameda Unified,5.2,43.5,38.3,9.3,-3.5,-12.7,english_only
171,61119,NaN,Alameda Unified,13.9,29.8,16.0,NaN,NaN,NaN,filipino
172,61119,NaN,Alameda Unified,NaN,NaN,NaN,2.3,-24.4,-26.7,foster_youth


In [128]:
dfe

,District Code,districtname,studentgroup,ela_currstatus,ela_priorstatus,ela_change,dfm
164,61119,Alameda Unified,african_american,-43.0,-48.7,5.7,NaN
165,61119,Alameda Unified,american_indian,-14.8,14.3,-29.1,NaN
166,61119,Alameda Unified,all_students,36.8,32.8,4.0,NaN
167,61119,Alameda Unified,asian,48.6,54.0,-5.3,NaN
168,61119,Alameda Unified,english_learners,-2.5,-1.4,-1.1,NaN
...,...,...,...,...,...,...,...
148816,72769,Wheatland Union High,pacific_islander,NaN,NaN,NaN,NaN
148817,72769,Wheatland Union High,rfep_only,NaN,NaN,NaN,NaN
148818,72769,Wheatland Union High,ses_disad,-28.4,-7.9,-20.6,NaN
148819,72769,Wheatland Union High,disabilities,-150.3,NaN,NaN,NaN


In [166]:
mscores = dfm.districtname, dfm.studentgroup, dfm.m_currstatus, dfm.m_priorstatus, dfm.m_change
mscores

(163            Alameda Unified
 164            Alameda Unified
 165            Alameda Unified
 166            Alameda Unified
 167            Alameda Unified
                   ...         
 148683    Wheatland Union High
 148684    Wheatland Union High
 148685    Wheatland Union High
 148686    Wheatland Union High
 148687    Wheatland Union High
 Name: districtname, Length: 13603, dtype: object,
 163       african_american
 164        american_indian
 165           all_students
 166                  asian
 167       english_learners
                 ...       
 148683    pacific_islander
 148684           rfep_only
 148685           ses_disad
 148686        disabilities
 148687               white
 Name: studentgroup, Length: 13603, dtype: object,
 163       -83.9
 164       -51.8
 165        14.3
 166        43.6
 167       -13.7
           ...  
 148683      NaN
 148684      NaN
 148685    -90.0
 148686   -226.6
 148687    -42.3
 Name: m_currstatus, Length: 13603, dtype: float64,

In [175]:
s2 = pandas.concat([dfe, dfm],
                   levels=[['studentgroup', 'districtname']],
                   axis=1)
s2

,District Code,districtname,studentgroup,ela_currstatus,ela_priorstatus,ela_change,dfm,District Code,districtname,studentgroup,m_currstatus,m_priorstatus,m_change
163,NaN,NaN,NaN,NaN,NaN,NaN,NaN,61119,Alameda Unified,african_american,-83.9,-82.8,-1.1
164,61119,Alameda Unified,african_american,-43.0,-48.7,5.7,NaN,61119,Alameda Unified,american_indian,-51.8,-50.0,-1.8
165,61119,Alameda Unified,american_indian,-14.8,14.3,-29.1,NaN,61119,Alameda Unified,all_students,14.3,10.8,3.4
166,61119,Alameda Unified,all_students,36.8,32.8,4.0,NaN,61119,Alameda Unified,asian,43.6,44.8,-1.2
167,61119,Alameda Unified,asian,48.6,54.0,-5.3,NaN,61119,Alameda Unified,english_learners,-13.7,-12.9,-0.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...
148816,72769,Wheatland Union High,pacific_islander,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
148817,72769,Wheatland Union High,rfep_only,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
148818,72769,Wheatland Union High,ses_disad,-28.4,-7.9,-20.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN
148819,72769,Wheatland Union High,disabilities,-150.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [170]:
s2 = pandas.concat([dfe, mscores],
                   axis=1,
                   levels=[['districtname', 'studentgroup',]],
                   ignore_index=True)
s2

TypeError: merge() got an unexpected keyword argument 'axis'

In [171]:
s1 = pandas.merge(dfe, dfm, how='inner', 
                  on=['District Code'], 
                  right_index=True)

TypeError: merge() got an unexpected keyword argument 'ignore_index'

In [126]:
s1
# english groups got messed up here

,District Code,districtname_x,studentgroup_x,ela_currstatus,ela_priorstatus,ela_change,districtname_y,studentgroup_y,m_currstatus,m_priorstatus,m_change
164,61119,Alameda Unified,african_american,-43.0,-48.7,5.7,Alameda Unified,african_american,-83.9,-82.8,-1.1
164,61119,Alameda Unified,african_american,-43.0,-48.7,5.7,Alameda Unified,american_indian,-51.8,-50.0,-1.8
164,61119,Alameda Unified,african_american,-43.0,-48.7,5.7,Alameda Unified,all_students,14.3,10.8,3.4
164,61119,Alameda Unified,african_american,-43.0,-48.7,5.7,Alameda Unified,asian,43.6,44.8,-1.2
164,61119,Alameda Unified,african_american,-43.0,-48.7,5.7,Alameda Unified,english_learners,-13.7,-12.9,-0.9
...,...,...,...,...,...,...,...,...,...,...,...
148820,72769,Wheatland Union High,white,11.7,14.2,-2.5,Wheatland Union High,pacific_islander,NaN,NaN,NaN
148820,72769,Wheatland Union High,white,11.7,14.2,-2.5,Wheatland Union High,rfep_only,NaN,NaN,NaN
148820,72769,Wheatland Union High,white,11.7,14.2,-2.5,Wheatland Union High,ses_disad,-90.0,-74.0,-16.0
148820,72769,Wheatland Union High,white,11.7,14.2,-2.5,Wheatland Union High,disabilities,-226.6,NaN,NaN


In [ ]:
s1.drop('districtname_y', axis=1, inplace=True)
s1.drop('studentgroup_y', axis=1, inplace=True)
s1 = s1.rename(columns={'districtname_x': 'districtname',
                          'studentgroup_x': 'studentgroup',
                        })

In [99]:
#s2 = pandas.merge(s1, dflcff, how='left', on=['District Code'])
#s2
# probably a better way to do this

,District Code,districtname,studentgroup,ela_currstatus,ela_priorstatus,ela_change,m_currstatus,m_priorstatus,m_change,Local Educational Agency,unduplicated_pupil_count,base_grant,supplemental_grant,concentration_grant,total_grants,cds
0,61119,Alameda Unified,african_american,-43.0,-48.7,5.7,-83.9,-82.8,-1.1,Alameda Unified,0.3486,"75,938,983","5,294,466",-,"82,158,116",01611190000000
1,61119,Alameda Unified,african_american,-43.0,-48.7,5.7,-51.8,-50.0,-1.8,Alameda Unified,0.3486,"75,938,983","5,294,466",-,"82,158,116",01611190000000
2,61119,Alameda Unified,african_american,-43.0,-48.7,5.7,14.3,10.8,3.4,Alameda Unified,0.3486,"75,938,983","5,294,466",-,"82,158,116",01611190000000
3,61119,Alameda Unified,african_american,-43.0,-48.7,5.7,43.6,44.8,-1.2,Alameda Unified,0.3486,"75,938,983","5,294,466",-,"82,158,116",01611190000000
4,61119,Alameda Unified,african_american,-43.0,-48.7,5.7,-13.7,-12.9,-0.9,Alameda Unified,0.3486,"75,938,983","5,294,466",-,"82,158,116",01611190000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
207554,72769,Wheatland Union High,white,11.7,14.2,-2.5,NaN,NaN,NaN,Wheatland Union High,0.4283,"7,062,422","604,967",-,"7,852,628",58727690000000
207555,72769,Wheatland Union High,white,11.7,14.2,-2.5,NaN,NaN,NaN,Wheatland Union High,0.4283,"7,062,422","604,967",-,"7,852,628",58727690000000
207556,72769,Wheatland Union High,white,11.7,14.2,-2.5,-90.0,-74.0,-16.0,Wheatland Union High,0.4283,"7,062,422","604,967",-,"7,852,628",58727690000000
207557,72769,Wheatland Union High,white,11.7,14.2,-2.5,-226.6,NaN,NaN,Wheatland Union High,0.4283,"7,062,422","604,967",-,"7,852,628",58727690000000


In [198]:
len(elat)

932

In [199]:
len(matht)

931

In [212]:
dflcff['District Code'].drop(, inplace=True)
dflcff

KeyError: "['61119' '61127' '61143' '61150' '61168' '61176' '61192' '61200' '61218'\n '61234' '61242' '61259' '61275' '61291' '61309' '75093' '75101' '75119'\n '61333' '73981' '61382' '61408' '61424' '61432' '61440' '61457' '61499'\n '61507' '61515' '61523' '61531' '61549' '73379' '75507' '61556' '61564'\n '61572' '61580' '61598' '61606' '61614' '61622' '61630' '61648' '61655'\n '61663' '61671' '61697' '61705' '61713' '61721' '61739' '61747' '61754'\n '61762' '61770' '61788' '61796' '61804' '61812' '61820' '61838' '61846'\n '61853' '61879' '61887' '61895' '61903' '61911' '61929' '61945' '61952'\n '61960' '61978' '61986' '73783' '61994' '62026' '62042' '62109' '62117'\n '62125' '62158' '62166' '62240' '62257' '62265' '62281' '62323' '62331'\n '62356' '62364' '62372' '62380' '62414' '62430' '62513' '62539' '62547'\n '73809' '73965' '73999' '75127' '75234' '75275' '75408' '75598' '76778'\n '62554' '62596' '62638' '62646' '62653' '62661' '75481' '76562' '62679'\n '62687' '62695' '62703' '6

In [165]:
dflcff.drop(s3, inplace=True)
dflcff

KeyError: "['75663' '31609' '76893' '75697' '31617' '77024' '77149' '77032' '77081'\n '76547' '76992' '76950' '76919' '75630' '77222' '76927' '77065' '31625'\n '77164' '76901' '77131' '77099' '77172' '76471' '77156' '77198' '77180'\n '77206' '77214' '76943' '77107' '32276' '76968' '76885'] not found in axis"

### play cells

In [24]:
i = ela.loc[ela['rtype'] == 'S']
o = i = pandas.concat([i.pop(x) for x in ['schoolname', 
                                      'cds']],
                  1)
o
# had to check this school's cds code to validate with another dataset

,schoolname,cds
30,Envision Academy for Arts & Technology,1100170112607
31,Envision Academy for Arts & Technology,1100170112607
32,Envision Academy for Arts & Technology,1100170112607
33,Envision Academy for Arts & Technology,1100170112607
34,Envision Academy for Arts & Technology,1100170112607
...,...,...
148833,Wheatland Union High,58727695838305
148834,Wheatland Union High,58727695838305
148835,Wheatland Union High,58727695838305
148836,Wheatland Union High,58727695838305


In [ ]:
mlist = []
for group in dfm['District Code']:
    mlist.append(dfm.studentgroup)
    for group in dfm['studentgroup']:
        mlist.append(dfm.m_currstatus)
mlist

In [ ]:
dfmm = dfm
dfm1 = pandas.concat([dfmm.pop(x) for x in ['District Code',
                                           'studentgroup',
                                           'currstatus',
                                           'priorstatus',
                                           'change',
                                          ]])

# Census Data

#### SCAG counties and their FIPS codes:
<br> Los Angeles (06037)<br> Orange (06059)<br> Riverside (06065)<br> San Bernardino (06071)<br> San Diego (06073)<br>Imperial (06025)<br> Ventura (06111)

In [2]:
cacom = Community.from_census(state_fips='06')

NameError: name 'Community' is not defined

In [ ]:
cacom.gdf['county'] = cacom.gdf.geoid.str[:5]
# create counties variable

In [ ]:
cacom.gdf.groupby('county').sum()
#group

In [26]:
scag = Community.from_census(county_fips=['06025', '06037', '06059', '06071', '06111', '06073', '06065'])

NameError: name 'Community' is not defined

In [27]:
rivco = Community.from_census(county_fips=['06065'])

NameError: name 'Community' is not defined

In [28]:
orangeco = Community.from_census(county_fips=['06059'])

NameError: name 'Community' is not defined

In [ ]:
laco = Community.from_census(county_fips=['06037'])

In [ ]:
sbco = Community.from_census(county_fips=['06071'])

In [ ]:
sdco = Community.from_census(county_fips=['06073'])

In [ ]:
f, ax = plt.subplots(1, figsize=(30, 30))
ax = rivco.gdf[rivco.gdf.year==2010].plot('p_poverty_rate_over_65', axes=ax)
plt.show()

# change gdf, variable, and year

### This 'plot' function will produce three maps, one from each of the census data years. We can use it to pass in communities and a specific variable to provide a preliminary view of changes over 30 years.

In [ ]:
def plot(community, column):
    fig, axs = plt.subplots(1,3, figsize=(20,20))
    axs=axs.flatten()
    
    community.gdf[community.gdf.year==1990].dropna(subset=[column]).plot(column=column, scheme='quantiles', cmap='Greens', k=7, ax=axs[0])
    axs[0].axis('off')
    axs[0].set_title('1990')

    community.gdf[community.gdf.year==2000].dropna(subset=[column]).plot(column=column, scheme='quantiles', cmap='Greens', k=7, ax=axs[1])
    axs[1].axis('off')
    axs[1].set_title('2000')

    community.gdf[community.gdf.year==2010].dropna(subset=[column]).plot(column=column, scheme='quantiles', cmap='Greens', k=7, ax=axs[2])
    axs[2].axis('off')
    axs[2].set_title('2010')

In [ ]:
datasets.codebook()

### Variables of interest from census:
#### economic indicators
median_household_income <br>
median_home_value OR median_contract_rent<br>
median_income_whitehh<br>
median_income_blackhh<br>
median_income_hispanichh<br>
n_edu_college_greater - persons with at least 4-year degree (also available as %)<br>
n_edu_hs_less - persons with less than high school (also available as %)<br>

#### demographic indicators
n_recent_immigrant_pop (within the last 10 years)<br>
n_limited_english OR n_other_language<br>
n_hispanic_persons<br>
n_black_persons<br>
n_white_persons

In [ ]:
from segregation.aspatial import GiniSeg
index = GiniSeg(scag.gdf, 'n_mexican_pop', 'n_total_pop')
type(index)

# MERGE SCHOOL DISTRICTS WITH CENSUS DATA

## Alternative community construction

Can also construct communities from longitudinal tract database (ltdb)

In [ ]:
datasets.msas()[datasets.msas().name.str.startswith('Riverside')]

In [ ]:
datasets.msas()[datasets.msas().name.str.startswith('Orange')]

In [ ]:
datasets.msas()[datasets.msas().name.str.startswith('San')]

In [ ]:
LA = Community.from_ltdb(msa_fips='31080')

In [ ]:
SD = Community.from_ltdb(msa_fips='41740')

In [ ]:
riverside = Community.from_ltdb(msa_fips='40140')